# Spatial Smoothing with Label Propagation

### Import packages enviroment and project path

In [ ]:
import pandas as pd
import pickle
import numpy as np
import geopy.distance
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from sklearn.semi_supervised import LabelPropagation
from tqdm import tqdm
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
Project_Path='Project/Path'

### Column names for processing

In [ ]:
Names=['Date_Sin','Covid','Holidays','Capacity','Week_Day_Sin','Month_Sin','Real_Time','temp','humidity','Γενικό Νοσοκομείο Θεσσαλονίκης «Γ. Γεννηματάς»', 'Λιμάνι','Δημαρχείο Θεσσαλονίκης','Λευκός Πύργος','Αγορά Καπάνι','Λαδάδικα','Πλατεία Άθωνος','Πλατεία Αριστοτέλους','Ροτόντα','Πλατεία Αγίας Σοφίας','Πλατεία Αντιγονιδών','Μουσείο Μακεδονικού Αγώνα','Πλατεία Ναυαρίνου','Πάρκο ΧΑΝΘ','Ιερός Ναός Αγίου Δημητρίου','ΔΕΘ','ΑΠΘ','Άγαλμα Ελευθερίου Βενιζέλου','Ρωμαϊκή Αγορά Θεσσαλονίκης']
Names2=['Slot_id','Key','Date_Sin','Covid','Holidays','Capacity','Week_Day_Sin','Month_Sin','Real_Time','Real_Rate','temp','humidity']
Names3=['Slot_id','Capacity','Γενικό Νοσοκομείο Θεσσαλονίκης «Γ. Γεννηματάς»', 'Λιμάνι' ,'Δημαρχείο Θεσσαλονίκης','Λευκός Πύργος','Αγορά Καπάνι','Λαδάδικα','Πλατεία Άθωνος','Πλατεία Αριστοτέλους','Ροτόντα','Πλατεία Αγίας Σοφίας','Πλατεία Αντιγονιδών','Μουσείο Μακεδονικού Αγώνα','Πλατεία Ναυαρίνου','Πάρκο ΧΑΝΘ','Ιερός Ναός Αγίου Δημητρίου','ΔΕΘ','ΑΠΘ','Άγαλμα Ελευθερίου Βενιζέλου','Ρωμαϊκή Αγορά Θεσσαλονίκης']
Names4=['Capacity','Γενικό Νοσοκομείο Θεσσαλονίκης «Γ. Γεννηματάς»', 'Λιμάνι' ,'Δημαρχείο Θεσσαλονίκης','Λευκός Πύργος','Αγορά Καπάνι','Λαδάδικα','Πλατεία Άθωνος','Πλατεία Αριστοτέλους','Ροτόντα','Πλατεία Αγίας Σοφίας','Πλατεία Αντιγονιδών','Μουσείο Μακεδονικού Αγώνα','Πλατεία Ναυαρίνου','Πάρκο ΧΑΝΘ','Ιερός Ναός Αγίου Δημητρίου','ΔΕΘ','ΑΠΘ','Άγαλμα Ελευθερίου Βενιζέλου','Ρωμαϊκή Αγορά Θεσσαλονίκης']
Names5=['Slot_id','Covid','Holidays','Capacity','temp','humidity','Date_Sin','Week_Day_Sin','Month_Sin','Real_Time','Γενικό Νοσοκομείο Θεσσαλονίκης «Γ. Γεννηματάς»', 'Λιμάνι' ,'Δημαρχείο Θεσσαλονίκης','Λευκός Πύργος','Αγορά Καπάνι','Λαδάδικα','Πλατεία Άθωνος','Πλατεία Αριστοτέλους','Ροτόντα','Πλατεία Αγίας Σοφίας','Πλατεία Αντιγονιδών','Μουσείο Μακεδονικού Αγώνα','Πλατεία Ναυαρίνου','Πάρκο ΧΑΝΘ','Ιερός Ναός Αγίου Δημητρίου','ΔΕΘ','ΑΠΘ','Άγαλμα Ελευθερίου Βενιζέλου','Ρωμαϊκή Αγορά Θεσσαλονίκης']
Names6=['Slot_id','Key','Date_Sin','Covid','Holidays','Capacity2','Week_Day_Sin','Month_Sin','Real_Time','Real_Rate','temp','humidity']
Names7=['Slot_id','Covid','Holidays','Capacity','temp','humidity','Date_Sin','Week_Day_Sin','Month_Sin','Real_Time','Γενικό Νοσοκομείο Θεσσαλονίκης «Γ. Γεννηματάς»', 'Λιμάνι' ,'Δημαρχείο Θεσσαλονίκης','Λευκός Πύργος','Αγορά Καπάνι','Λαδάδικα','Πλατεία Άθωνος','Πλατεία Αριστοτέλους','Ροτόντα','Πλατεία Αγίας Σοφίας','Πλατεία Αντιγονιδών','Μουσείο Μακεδονικού Αγώνα','Πλατεία Ναυαρίνου','Πάρκο ΧΑΝΘ','Ιερός Ναός Αγίου Δημητρίου','ΔΕΘ','ΑΠΘ','Άγαλμα Ελευθερίου Βενιζέλου','Ρωμαϊκή Αγορά Θεσσαλονίκης','Predictions','Target']
Names9=['Slot_id','Date_Sin','Week_Day_Sin','Month_Sin','Real_Time','Γενικό Νοσοκομείο Θεσσαλονίκης «Γ. Γεννηματάς»', 'Λιμάνι' ,'Δημαρχείο Θεσσαλονίκης','Λευκός Πύργος','Αγορά Καπάνι','Λαδάδικα','Πλατεία Άθωνος','Πλατεία Αριστοτέλους','Ροτόντα','Πλατεία Αγίας Σοφίας','Πλατεία Αντιγονιδών','Μουσείο Μακεδονικού Αγώνα','Πλατεία Ναυαρίνου','Πάρκο ΧΑΝΘ','Ιερός Ναός Αγίου Δημητρίου','ΔΕΘ','ΑΠΘ','Άγαλμα Ελευθερίου Βενιζέλου','Ρωμαϊκή Αγορά Θεσσαλονίκης','Predictions']
Names_LP=['Date_Sin','Covid','Holidays','Capacity','Week_Day_Sin','Month_Sin','Real_Time','temp','humidity','Γενικό Νοσοκομείο Θεσσαλονίκης «Γ. Γεννηματάς»', 'Λιμάνι','Δημαρχείο Θεσσαλονίκης','Λευκός Πύργος','Αγορά Καπάνι','Λαδάδικα','Πλατεία Άθωνος','Πλατεία Αριστοτέλους','Ροτόντα','Πλατεία Αγίας Σοφίας','Πλατεία Αντιγονιδών','Μουσείο Μακεδονικού Αγώνα','Πλατεία Ναυαρίνου','Πάρκο ΧΑΝΘ','Ιερός Ναός Αγίου Δημητρίου','ΔΕΘ','ΑΠΘ','Άγαλμα Ελευθερίου Βενιζέλου','Ρωμαϊκή Αγορά Θεσσαλονίκης','Real_Rate']

In [ ]:
def save_object(obj, filename):
    with open(filename, 'wb') as outp:
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)

# Prepare Data For Graph

### Main preparation

In [ ]:
def Prepare_Initial_Data(Data,Final_Weather_Data,Distance_Data): 
    Data['Time_Distance']=0
    Data=Data.drop(['Time_Int'], axis=1)
    a=Data['Slot_Timeint']
    b=Data['Ilegality_Rate']
    Data=Data.drop(['Slot_Timeint'], axis=1)
    Data=Data.drop(['Ilegality_Rate'], axis=1)
    Data.insert(9, "Real_Time", a, True)
    Data.insert(10, "Real_Rate", b, True) 
    Data=pd.merge(Data, Final_Weather_Data, on='Key')
    Data=pd.merge(Data, Distance_Data, on='Slot_id')
    return Data

In [ ]:
def Prepare_Dataset(Data,Final_Weather_Data,Distance_Data): 
    Data=Data.drop(['Time_Int'], axis=1)
    a=Data['Slot_Timeint']
    b=Data['Ilegality_Rate']
    Data=Data.drop(['Slot_Timeint'], axis=1)
    Data=Data.drop(['Ilegality_Rate'], axis=1)
    Data.insert(8, "Real_Time", a, True)
    Data.insert(9, "Real_Rate", b, True)       
    Data=pd.merge(Test, Final_Weather_Data, on='Key')
    Data=pd.merge(Data, Distance_Df, on='Slot_id')
    return Data

### Scalling X Standardization

In [ ]:
def Scaller_For_Prediction(Data):
    Slot_id=Data['Slot_id']
    Key=Data['Key']
    Real_Rate=Data['Real_Rate']
    
    Data=Data.drop(['Slot_id'], axis=1)
    Data=Data.drop(['Key'], axis=1)
    Data=Data.drop(['Real_Rate'], axis=1)
    
    Standar_Scaller = pickle.load(open(Project_Path+ '/Standar_Scaller2.pkl','rb'))
    Data = Standar_Scaller.transform(Data)
    Data=pd.DataFrame(Data,columns =Names)
    
    Data.insert(0, "Slot_id", Slot_id, True)
    Data.insert(1, "Key", Key, True)
    Data.insert(10, "Real_Rate", Real_Rate, True)
    
    return Data

In [ ]:
def Scaller_For_Graph(Train,Test):
    Slot_id_Train=Train['Slot_id']
    Key_Train=Train['Key']
    Real_Rate_Train=Train['Real_Rate']
    
    Train=Train.drop(['Slot_id'], axis=1)
    Train=Train.drop(['Key'], axis=1)
    Train=Train.drop(['Real_Rate'], axis=1)
    
    Slot_id_Test=Test['Slot_id']
    Key_Test=Test['Key']
    Real_Rate_Test=Test['Real_Rate']
    
    Test=Test.drop(['Slot_id'], axis=1)
    Test=Test.drop(['Key'], axis=1)
    Test=Test.drop(['Real_Rate'], axis=1)
    
    Standar_Scaller = StandardScaler()
    Train=Standar_Scaller.fit_transform(Train)
    Test = Standar_Scaller.transform(Test)
    
    Train=pd.DataFrame(Train,columns = Names)
    Test=pd.DataFrame(Test,columns = Names)

    Train.insert(0, "Slot_id", Slot_id_Train, True)
    Train.insert(1, "Key", Key_Train, True)
    Train.insert(10, "Real_Rate", Real_Rate_Train, True)
    
    Test.insert(0, "Slot_id", Slot_id_Test, True)
    Test.insert(1, "Key", Key_Test, True)
    Test.insert(10, "Real_Rate", Real_Rate_Test, True)
    
    return Train,Test

In [ ]:
def Scalled_Distances(Data):
    Distance_Data=Data
    Distance_Data=pd.DataFrame(Distance_Data)
    Distance_Data=Distance_Data[Names3]
    Distance_Data.drop_duplicates(subset ="Slot_id",keep ='first', inplace = True)
    Distance_Data=Distance_Data.reset_index()
    Distance_Data=Distance_Data.drop(['index'], axis=1)
    return Distance_Data  

### Create graphs from initial dataset

In [ ]:
def Get_Unique_Time_Id(Data):
    Data=Data.sort_values(by=['Key'])
    Time_Id=Data['Key'].unique()
    return Time_Id   

In [ ]:
def Delete_Slot_Info(Data):
    Data=Data[Names2]
    return Data

In [ ]:
def Get_Subsets_and_Lenght(Data,Time_Id):
    Subsets=[]
    Subsets=[0 for i in range(len(Time_Id))]
    Length=[]
    Length=[0 for i in range(len(Time_Id))]
    Mask=[0 for i in range(len(Time_Id))]

    for i in tqdm( range (0,len(Time_Id))):
        Subsets[i] = Data[Data["Key"]==Time_Id[i]]
        Length[i] = len(Subsets[i])
        Mask[i] = Subsets[i]['Slot_id'].values.tolist()
    return Subsets,Length,Mask

In [ ]:
def Fill_Subsets(Subsets,Time_Id,Length,Legal_Ilegan):
    Sectors=Legal_Ilegan['Slot_id'].unique()
    Sectors=sorted(Sectors)
    Datasets=[]
    Datasets = [0 for i in range(len(Time_Id))]

    for i in tqdm(range (0,len(Subsets))):
        S=[]
        List=[]
        Train_List=Subsets[i].values.tolist()
        for j in range (0,Length[i]):
            S.append(Train_List[j][0])
            List.append(Train_List[j])

        for k in range (0,len(Sectors)):
            List2=[]
            if Sectors[k] not in S:
                List2.append(int(Sectors[k]))

                for l in range (1,12):
                    List2.append(Train_List[0][l])
                List.append(List2)

        Datasets[i]=pd.DataFrame(List, columns=[Names6])
        Datasets[i].Slot_id = Datasets[i].Slot_id.astype(int)
    return Datasets
    

In [ ]:
def Get_Distances(Datasets,Time_Id,Distance_Data):
    Distance_Data_List=Distance_Data.values.tolist()
    for i in tqdm(range(0,len(Time_Id))):
        Dataset_List=Datasets[i].values.tolist()
        Distances=[]
        Distances_Df=[]
        for j in range (0,len(Dataset_List)):
            for k in range(0,len(Distance_Data_List)):
                if Dataset_List[j][0]==Distance_Data_List[k][0]:
                    Distances.append(Distance_Data_List[k][1:])
                    break
        Distances_Df=pd.DataFrame(Distances, columns=[Names4])
        Datasets[i] = pd.concat([Datasets[i], Distances_Df], axis=1)

    for i in tqdm(range (0,len(Time_Id) )):
        Datasets[i]=Datasets[i].drop(['Capacity2'], axis=1)
        Datasets[i].insert(5, 'Capacity', Datasets[i].pop('Capacity'))

    return Datasets

### Prediction using the DNN model

In [ ]:
def Make_Predictions(Datasets,Time_Id,Length):

    Predictions_Df=[]
    Predictions_Df=[0 for i in range(len(Datasets))] 
    Real_Target=[]
    Real_Target=[0 for i in range(len(Datasets))] 

    for i in tqdm(range (0,len(Time_Id))):
        Real_Target[i]=np.array(Datasets[i]['Real_Rate'])

        Feautures = Datasets[i][Names]
        Test=np.array(Feautures)   
        Predictions=model.predict([Test])
        Predictions_Df[i]=Predictions

    for i in range (0,len(Time_Id)):
        for j in range (Length[i],222):
            Real_Target[i][j]=Predictions_Df[i][j]
    return Real_Target,Predictions_Df


### Final graph object

In [ ]:
def Get_Final_Dataset(Datasets,Real_Target,Predictions_Df,Time_Id):
    Final_Dataset=[]
    Final_Dataset=[0 for i in range(len(Time_Id))] 
    for i in tqdm(range (0,len(Time_Id))):
        Predictions_Df[i]=pd.DataFrame(Predictions_Df[i],columns=['Predictions'])
        Real_Target[i]=pd.DataFrame(Real_Target[i],columns=['Target'])
        Final_Dataset[i]=pd.concat([Datasets[i][Names5],Predictions_Df[i],Real_Target[i]], axis=1)
        Final_Dataset[i].columns=Names7
    return Final_Dataset
    

### Combine all functions

In [ ]:
def Prepare_Data_For_Predictions(test_data,Distance_Data,Legal_Ilegan):
    Time_Id=Get_Unique_Time_Id(test_data)
    test_data=Delete_Slot_Info(test_data)
    Subsets,Length,Mask=Get_Subsets_and_Lenght(test_data,Time_Id)
    Datasets=Fill_Subsets(Subsets,Time_Id,Length,Legal_Ilegan)
    Datasets=Get_Distances(Datasets,Time_Id,Distance_Data)
    Real_Target,Predictions_Df=Make_Predictions(Datasets,Time_Id,Length)
    return Real_Target,Predictions_Df

In [ ]:
def Prepare_Data(train_data,Distance_Data,Legal_Ilegan,Real_Target,Predictions_Df):
    Time_Id=Get_Unique_Time_Id(train_data)
    train_data=Delete_Slot_Info(train_data)
    Subsets,Length,Mask=Get_Subsets_and_Lenght(train_data,Time_Id)
    Datasets=Fill_Subsets(Subsets,Time_Id,Length,Legal_Ilegan)
    Datasets=Get_Distances(Datasets,Time_Id,Distance_Data)    
    Train_Dataset=Get_Final_Dataset(Datasets,Real_Target,Predictions_Df,Time_Id)
    return Train_Dataset,Length,Mask,Time_Id

## Temporal Signal Creation

### Data loading and initial preprocessing

In [ ]:
Distance_Data=pd.read_csv(Project_Path+ '/Data/Distance.csv',sep=',',index_col=0)
Final_Weather_Data=pd.read_csv(Project_Path+ '/Data/Final_Weather_Data.csv',low_memory=False,sep=',',index_col=0)
train_data=pd.read_csv(Project_Path+ '/Data/Train_TimeSeries.csv',sep=',',index_col=0)
test_data=pd.read_csv(Project_Path+ '/Data/Test_TimeSeries.csv',sep=',',index_col=0)

TestDF=Prepare_Initial_Data(test_data,Final_Weather_Data,Distance_Data) #No smoothing
TrainDF=Prepare_Initial_Data(train_data,Final_Weather_Data,Distance_Data)

TrainDF=TrainDF.drop(['Time_Distance'], axis=1)
TestDF=TestDF.drop(['Time_Distance'], axis=1)

TrainDF.to_csv(Project_Path+ '/Data/Raw_Train_TimeSeries.csv')
TestDF.to_csv(Project_Path+ '/Data/Raw_Test_TimeSeries.csv')

TrainDF=TrainDF.drop(['Slot_id'], axis=1)
TrainDF=TrainDF.drop(['Key'], axis=1)

train_targets=TrainDF['Real_Rate']
train_data=TrainDF.drop(['Real_Rate'], axis=1)

Standar_Scaller = StandardScaler()
train_data=Standar_Scaller.fit_transform(train_data)

with open(Project_Path+'/Standar_Scaller2.pkl', 'wb') as f:
    pickle.dump(Standar_Scaller, f,  protocol=2)

In [ ]:
model = keras.models.load_model(Project_Path+ '/DNN_Regressor')
model.compile(optimizer='adamax', loss='MSE', metrics=['MAE'])

Legal_Ilegan=pd.read_csv(Project_Path+ '/Data/Full_TimeSeries.csv',sep=',',index_col=0)
test_data=pd.read_csv(Project_Path+ '/Data/Raw_Test_TimeSeries.csv',sep=',',index_col=0)
train_data=pd.read_csv(Project_Path+ '/Data/Raw_Train_TimeSeries.csv',sep=',',index_col=0)

In [ ]:
Legal_Ilegan=Prepare_Dataset(Legal_Ilegan,Final_Weather_Data,Distance_Data)
test_data1=Scaller_For_Prediction(test_data)
train_data1=Scaller_For_Prediction(train_data)
Legal_Ilegan1=Scaller_For_Prediction(Legal_Ilegan)
Distance_Data_For_Predictions=Scalled_Distances(Legal_Ilegan1)
train_data,test_data=Scaller_For_Graph(train_data,test_data)
Distance_Data=Scalled_Distances(Legal_Ilegan1)

### Trainset

In [ ]:
Real_Target_Train,Predictions_Df_Train=Prepare_Data_For_Predictions(train_data1,Distance_Data_For_Predictions,Legal_Ilegan)
Train_Dataset,Length_Train,Train_Mask,Train_Date=Prepare_Data(train_data,Distance_Data,Legal_Ilegan,Real_Target_Train,Predictions_Df_Train)

save_object(Train_Dataset, Project_Path+'/Data/Dataset_Graph_For_LP.pkl')
with open(Project_Path+'/Data/Length_RawTrain.txt', "w") as file:
    file.write(str(Length_Train))

### Train mask

In [ ]:
Slots=Train_Dataset[0]['Slot_id']
Slots=sorted(Slots)

Train_Binary_Mask=[]
for j in range(0,len(Train_Mask)):
    a=[]
    for i in range (0,len(Slots)):
        a.append(Slots[i] in (Train_Mask[j]))
    Train_Binary_Mask.append(a)

Train_Binary_Mask=pd.DataFrame(Train_Binary_Mask,columns=Slots,index=Train_Date)
Train_Binary_Mask=Train_Binary_Mask.T
Train_Binary_Mask = Train_Binary_Mask.replace(False, 0)
Train_Binary_Mask = Train_Binary_Mask.replace(True, 1)
Train_Binary_Mask.to_csv(Project_Path+ '/Data/Raw_Train_Mask.csv')
Train_Binary_Mask

## Label Propagation model

In [ ]:
Sectors=pd.read_csv(Project_Path+ '/Data/Parking_Slot_Proccesed.csv',sep=',', index_col=0)
Nodelist=Sectors[['Slot_id','Latitude','Longitude']]
Legal_Ilegan2=pd.read_csv(Project_Path+ '/Data/Scan_Data_Reg_2.3.csv',sep=',',index_col=0)
Unique_Sectors=Legal_Ilegan2['Slot_id'].unique()
Unique_Sectors=pd.DataFrame(Unique_Sectors, columns=['Slot_id'])
Nodelist=pd.merge(Unique_Sectors,Nodelist, on='Slot_id')
Nodelist['Slot_id']=sorted(Nodelist['Slot_id'])

### Find nearest unlabeled nodes

In [ ]:
List=Nodelist.values.tolist()
Distance=[]
for i in range (0,len(List)):
    Distance2=[]
    for j in range (0,len(List)):
        d= geopy.distance.geodesic((List[i][1],List[i][2]), (List[j][1],List[j][2])).m
        Distance2.append(d)
    Distance.append(Distance2)
Names=Nodelist['Slot_id'].values.tolist()
Distance=pd.DataFrame(Distance, columns=Names, index=Names)

Columns=Distance.columns
NearSectors=[]
for i in tqdm(range(0,len(Columns))):
    Nearest=Distance.nsmallest(3,Columns[i] )
    Nearest=Nearest[Columns[i]]
    Nearest=Nearest.reset_index(level=0)
    Nearest=Nearest.values.tolist()
    NearSectors.append(Nearest)

In [ ]:
def Get_Target_Sectros(Train_Dataset,NearSectors):
    Sectors_List=[]
    for k in tqdm(range (0,len(Train_Dataset))):
        Existing_Sectors=Train_Dataset[k]['Slot_id'].values.tolist()
        ls=[]
        for i in range (0,Length_Train[k]):
            ls.append(Existing_Sectors[i])
            for j in range (0,len(NearSectors)):
                if Existing_Sectors[i]==int(NearSectors[j][0][0]): 
                    if int(NearSectors[j][1][0]) not in Existing_Sectors[0:Length_Train[k]]:
                        ls.append(int(NearSectors[j][1][0]))
                    if NearSectors[j][2][0] not in Existing_Sectors[0:Length_Train[k]]:
                        ls.append(int(NearSectors[j][2][0]))
        x= np.array(ls)
        ls=np.unique(x)       
        Sectors_List.append(ls)
    return Sectors_List        

In [ ]:
Sectors_List=Get_Target_Sectros(Train_Dataset,NearSectors)

In [ ]:
for i in tqdm(range (0,len(Train_Dataset))):
    T=[]
    ls=Train_Dataset[i]['Slot_id'].values.tolist()
    for j in range (0,len(ls)):
        if ls[j] not in Sectors_List[i]:
            T.append(False)
        else:
            T.append(True)
    Train_Dataset[i]['Flag']=T 

In [ ]:
save_object(Train_Dataset, Project_Path+ '/Data/Dataset_Graph_For_LP.pkl')

## Data Preprocessing for Label Propagation Model

In [ ]:
with open(Project_Path+'/Data/Dataset_Graph_For_LP.pkl', 'rb') as inp:
    Train_Dataset = pickle.load(inp)

for i in range (0,len(Train_Dataset)):
    Train_Dataset[i]['Target'].mask(Train_Dataset[i]['Target'] == Train_Dataset[i]['Predictions'], -1, inplace=True)
    Train_Dataset[i]=Train_Dataset[i].loc[Train_Dataset[i]['Flag'] == True]
    
Dataset = pd.concat(Train_Dataset, axis=0)
Dataset['Real_Rate']=Dataset['Target']
TrainDF=Dataset[Names_LP]

### Convert regression to classification dataset

In [ ]:
TrainDF['Real_Rate']=round(TrainDF['Real_Rate'],2)
TrainDF['Real_Rate']=TrainDF['Real_Rate']*100
TrainDF['Real_Rate']=TrainDF['Real_Rate'].astype(int)
TrainDF['Real_Rate'].mask(TrainDF['Real_Rate'] == -100, -1, inplace=True)

In [ ]:
Data=TrainDF
y=Data['Real_Rate']
X=Data.drop(['Real_Rate'], axis=1)

In [ ]:
Rates=Data['Real_Rate'].unique()
Rates=Rates/100
Rates=sorted(Rates)
Rates=Rates[1:]
len(Rates)

## Train and Save Model

In [ ]:
model = LabelPropagation(kernel='knn',n_neighbors=301, max_iter=1000,n_jobs=-1)
model.fit(X, y)

In [ ]:
filename = Project_Path +'/LabelPropagation.sav'
pickle.dump(model, open(filename, 'wb'))